# INTRODUCTION

## Abstract

**Keyword** : **Time series**, **LSTM**, **Prediction water level**

To achieve this of Acea, we have used various time series analysis and machine learning techniques. In this work, we developed a prediction model based on LSTM (Long short term memory),and prophet, one of the artificial neural network (ANN) algorithms, for predicting each waterbody.

Having approched this subject, we have concerned about each characteristic et geological dataset. Our model was developed by Keras on python-based work.

## Methodology
First, let's take a overview about each waterbody dataset doing Exploratory data analysis(including data description, histogram, plotting). And we will an insight which give us a good approach to modeling.
And handling missing data, dataset will be conserved maximum data to model.
Next, we will develope predictive models and tune these models.

# EXPLORATORY DATA ANALYSIS

## Module import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import math

# Geographic
from sklearn import preprocessing
import matplotlib.cm as cm
from geopy.geocoders import Nominatim
import folium
from tqdm import tqdm 

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, date

# Visualization 
import missingno as msno # missing values
import matplotlib.pyplot as plt
import seaborn as sns # Visualization
from matplotlib.pyplot import figure # Visualization

In [ ]:
import tensorflow as tf
import random

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import gc
import lightgbm as lgb

In [ ]:
def plot_corr_hitmap(df, title):    
    fig, ax = plt.subplots(figsize=(20,10)) 
    ax.set_title(f'Correlation Matrix - {title}', fontsize=14)
    ax = sns.heatmap(df.corr(method='pearson'), vmin=-1, vmax=1, center=0, cmap='Blues', annot=True, square=False)

## Data import
Import each waterbody data set( `AQUIFER`, `WATER SPRING`, `RIVER`, `LAKE`) 

### AQUIFER

In [ ]:
path = '/kaggle/input/acea-water-prediction/'

df_auser = pd.read_csv(f'{path}Aquifer_Auser.csv')
df_doganella = pd.read_csv(f'{path}Aquifer_Doganella.csv')
df_luco = pd.read_csv(f'{path}Aquifer_Luco.csv')
df_petrignano = pd.read_csv(f'{path}Aquifer_Petrignano.csv')

auser = df_auser.copy()
doganella = df_doganella.copy()
luco = df_luco.copy()
petrignano = df_petrignano.copy()

In [ ]:
# 'Date' : Series -> datetime 

auser['Date']=pd.to_datetime(auser['Date'],format ='%d/%m/%Y')
doganella['Date']=pd.to_datetime(doganella['Date'],format ='%d/%m/%Y')
luco['Date']=pd.to_datetime(luco['Date'],format ='%d/%m/%Y')
petrignano['Date']=pd.to_datetime(petrignano['Date'],format ='%d/%m/%Y')

### WATER SPRING

In [ ]:
path = '/kaggle/input/acea-water-prediction/'

df_amiata = pd.read_csv(f'{path}Water_Spring_Amiata.csv')
df_lupa = pd.read_csv(f'{path}Water_Spring_Lupa.csv')
df_madonna = pd.read_csv(f'{path}Water_Spring_Madonna_di_Canneto.csv')

amiata = df_amiata.copy()
lupa = df_lupa.copy()
madonna = df_madonna.copy()

In [ ]:
# 'Date' : Series -> datetime 

amiata['Date']=pd.to_datetime(amiata['Date'],format ='%d/%m/%Y')
lupa['Date']=pd.to_datetime(lupa['Date'],format ='%d/%m/%Y')
madonna['Date']=pd.to_datetime(madonna['Date'],format ='%d/%m/%Y')

### RIVER

In [ ]:
path = '/kaggle/input/acea-water-prediction/'

df_arno = pd.read_csv(f'{path}River_Arno.csv')

arno = df_arno.copy()

In [ ]:
# 'Date' : Series -> datetime 

arno['Date']=pd.to_datetime(arno['Date'],format ='%d/%m/%Y')

### LAKE

In [ ]:
path = '/kaggle/input/acea-water-prediction/'

df_bilancino = pd.read_csv(f'{path}Lake_Bilancino.csv')

bilancino = df_bilancino.copy()

In [ ]:
# 'Date' : Series -> datetime 

bilancino['Date']=pd.to_datetime(bilancino['Date'],format ='%d/%m/%Y')

## Data Description

### AQUIFER
Aquifer have 5 types of data (Rainfall, Depth_to_groundwater, temperature, volume, hydrometry) 

- Rainfall : quantity of rain falling(mm)
- Depth_to_groundwater: from ground floor to the groundwater level(m)
- Temperature : the temperature(°C)
- Volume : volume of water taken by the drinkin water treatment plant (m³)
- Hydrometry : the groundwater level (m)

#### View Variables

In [ ]:
# List of Dataframes columns

columns_auser = auser.columns.tolist()
columns_doganella = doganella.columns.tolist()
columns_luco = luco.columns.tolist()
columns_petrignano = petrignano.columns.tolist()

##### Auser

Aquifer Auser waterbody consists of two subsystems (NORH, SOUTH), where the former partly influences the behaviour of the latter. The levels of the NORTH sector are represented by the values of the SAL, PAG, CoS and DIEC wells, while the levels of the SOUTH sector by the LT2 well.

In [ ]:
# Number of variables
print('Number of variables : ' + str(len(columns_auser)))
print()

# Type of variables
print('type of variables : ')
print(auser.dtypes)

In [ ]:
print('Auser')
df_auser.describe()

It can be estimated that there are missing values in the number of counts and skewness. **Range of Rainfall** data is very wide.

##### Doganella

In [ ]:
print('Number of variables :', str(len(columns_doganella)))
print()
print('type of variables : ')
print(doganella.dtypes)

In [ ]:
print('Doganella')
doganella.describe()

It can be estimated that there are missing values in the number of counts and skewness. `Rainfall` data is very wide. `Temperature` data follows narmal distribution. 

##### Luco

In [ ]:
print('Number of variables :', str(len(columns_luco)))
print()
print('type of variables : ')
print(luco.dtypes)

In [ ]:
print('Luco')
luco.describe()

Dataset have similar data type. 

##### Petrignano

In [ ]:
print('Number of variables :', str(len(columns_petrignano)))
print('type of variables : ')
print(petrignano.dtypes)

In [ ]:
print('Petrignano')
petrignano.describe()

#### Histogram
In the preceding work, we were able to predict **skweness of distribution**. Histogram need to understand the data distibution of variables.

**Process**

1. divide dataframes into feuture variables and target variables
2. do histogram each variables

##### Auser

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_auser = auser[[target for target in auser.columns if target.startswith('Depth')]]
target_auser_list = target_auser.columns.tolist()
print('Number of Target : ' + str(len(target_auser_list)))
print(target_auser_list)

print()

# Feature variables
feature_auser = auser[[feature for feature in auser.columns if feature not in target_auser]]
feature_auser_list = feature_auser.columns.tolist()
print('Number of Features : ' + str(len(feature_auser_list)))
print(feature_auser_list)

In [ ]:
## HISTOGRAM (AUSER FEATURE VARIABLES)

# Number of columns
ncols = 3
# Number of rows
import math
nrows = math.ceil((len(feature_auser.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
#[sns.histplot(feature_auser.columns[i+1], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(auser_feature.columns)-1)]
[sns.distplot(feature_auser[feature_auser.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_auser.columns)-1)]

print('### AUSER_FEATURES ###')

In [ ]:
## HISTOGRAM (AUSER TARGET VARIABLES)

f, ax = plt.subplots(ncols = 5, nrows = 1, figsize= (24, 5))

for i in range(len(target_auser.columns)):
    sns.distplot(target_auser[target_auser.columns[i]], ax=ax[i])


print('### AUSER_TARGET ###')

##### Doganella

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_doganella = doganella[[target for target in doganella.columns if target.startswith('Depth')]]
target_doganella_list = target_doganella.columns.tolist()
print('Number of Target : ' + str(len(target_doganella_list)))
print(target_doganella_list)

print()

# Feature variables
feature_doganella = doganella[[feature for feature in doganella.columns if feature not in target_doganella]]
feature_doganella_list = feature_doganella.columns.tolist()
print('Number of Features : ' + str(len(feature_doganella_list)))
print(feature_doganella_list)

In [ ]:
# Number of columns
ncols = 3
# Number of rows
import math
nrows = math.ceil((len(feature_doganella.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
#[sns.histplot(auser_feature.columns[i+1], ax=axs[grid[i][0], grid[i][1]]) for i in range(0,len(auser_feature.columns)-1)]
[sns.distplot(feature_doganella[feature_doganella.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_doganella.columns)-1)]

print('### DOGANELLA_FEATURES ###')

In [ ]:
# Number of columns
ncols = 3
# Number of rows
import math
nrows = math.ceil((len(target_doganella.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(target_doganella[target_doganella.columns[i]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(target_doganella.columns)-1)]

print('### DOGANELLA_TARGET ###')

##### Luco

In [ ]:
# Target
target_luco = luco[[target for target in luco.columns if target.startswith('Depth')]]
target_luco_list = target_luco.columns.tolist()
print('Number of Target : ' + str(len(target_luco_list)))
print(target_luco_list)

print()

# Feature
feature_luco = luco[[feature for feature in luco.columns if feature not in target_luco]]
feature_luco_list = feature_luco.columns.tolist()
print('Number of Features : ' + str(len(feature_luco_list)))
print(feature_luco_list)

In [ ]:
# Number of columns
ncols = 3
# Number of rows
import math
nrows = math.ceil((len(feature_luco.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(feature_luco[feature_luco.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_luco.columns)-1)]

print('### LUCO_FEATURE ###')

In [ ]:
f, ax = plt.subplots(ncols = 4, nrows = 1, figsize= (24, 5))

for i in range(len(target_luco.columns)):
    sns.distplot(target_luco[target_luco.columns[i]], ax=ax[i])


print('###LUCO_TARGET ###')

##### Petrignano

In [ ]:
# Target
target_petrignano = petrignano[[target for target in petrignano.columns if target.startswith('Depth')]]
target_petrignano_list = target_petrignano.columns.tolist()
print('Number of Target : ' + str(len(target_petrignano_list)))
print(target_petrignano_list)

print()

# Feature
feature_petrignano = petrignano[[feature for feature in petrignano.columns if feature not in target_luco]]
feature_petrignano_list = feature_petrignano.columns.tolist()
print('Number of Features :' + str(len(feature_petrignano_list)))
print(feature_petrignano_list)

In [ ]:
# Number of columns
ncols = 3
# Number of rows
import math
nrows = math.ceil((len(feature_petrignano.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(feature_petrignano[feature_petrignano.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_petrignano.columns)-1)]

print('### PETRIGNANO_FEATURE ###')

In [ ]:
f, ax = plt.subplots(ncols = 2, nrows = 1, figsize= (24, 5))

for i in range(len(target_petrignano.columns)):
    sns.distplot(target_petrignano[target_petrignano.columns[i]], ax=ax[i])


print('### PETRIGNANO_TARGET ###')

#### Plotting

##### Auser

In [ ]:
feature_auser_list

In [ ]:
columns = feature_auser.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(auser['Date'], auser[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
target_auser_list

In [ ]:
columns = target_auser.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns), figsize= (10, 5 * len(columns)))

for var in range(0,len(columns)):
    axs[var].plot(auser['Date'], auser[columns[var]])
    
    axs[var].set_xlabel('Date')
    axs[var].set_ylabel(columns[var], fontsize = 15)
    
    axs[var].grid()

##### Doganella

In [ ]:
feature_doganella_list

In [ ]:
columns = feature_doganella.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(doganella['Date'], doganella[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
target_doganella_list

In [ ]:
columns = target_doganella.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns), figsize= (10, 5 * len(columns)))

for var in range(0,len(columns)):
    axs[var].plot(doganella['Date'], doganella[columns[var]])
    
    axs[var].set_xlabel('Date')
    axs[var].set_ylabel(columns[var], fontsize = 15)
    
    axs[var].grid()

##### Luco

In [ ]:
feature_luco_list

In [ ]:
columns = feature_luco.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(luco['Date'], luco[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
columns = target_luco.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(luco['Date'], luco[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

##### Petrignano

In [ ]:
columns = feature_petrignano.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(petrignano['Date'], petrignano[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
columns = target_petrignano.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns), figsize= (10, 5 * len(columns)))

for var in range(0,len(columns)):
    axs[var].plot(petrignano['Date'], petrignano[columns[var]])
    
    axs[var].set_xlabel('Date')
    axs[var].set_ylabel(columns[var], fontsize = 15)
    
    axs[var].grid()

### WATER SPRING

#### View Variables

In [ ]:
# List of Dataframes columns

columns_amiata = amiata.columns.tolist()
columns_lupa = lupa.columns.tolist()
columns_madonna = madonna.columns.tolist()

##### Amiata

Amiata is reached through the Ermicciolo, Arbure, Bugnano and Galleria Alta springs. The levels and volumes of the four springs are influenced by the parameters: rainfall, depth to groundwater, temperature.

In [ ]:
print('Number of variables : ' + str(len(columns_amiata)))

print('type of variables : ')
print(amiata.dtypes)

In [ ]:
print('Amiata')
amiata.describe()

##### Lupa

In [ ]:
print('Number of variables : ' + str(len(columns_lupa)))

print('type of variables : ')
print(lupa.dtypes)

In [ ]:
print('Lupa')
lupa.describe()

##### Madonna di Canneto

In [ ]:
print('Number of variables : ' + str(len(columns_madonna)))

print('type of variables : ')
print(madonna.dtypes)

In [ ]:
print('Madonna')
df_auser.describe()

#### Histogram

##### Amiata

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_amiata = amiata[[target for target in amiata.columns if target.startswith('Flow')]]
target_amiata_list = target_amiata.columns.tolist()
print('Number of Target : ' + str(len(target_amiata_list)))
print(target_amiata_list)

print()

# Feature variables
feature_amiata = amiata[[feature for feature in amiata.columns if feature not in target_amiata]]
feature_amiata_list = feature_amiata.columns.tolist()
print('Number of Features : ' + str(len(feature_amiata_list)))
print(feature_amiata_list)

In [ ]:
## HISTOGRAM (AMIATA FEATURE VARIABLES)

# Number of columns
ncols = 4
nrows = math.ceil((len(feature_amiata.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(feature_amiata[feature_amiata.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_amiata.columns)-1)]

print('### AMIATA_FEATURES ###')

In [ ]:
## HISTOGRAM (AMIATA TARGET VARIABLES)

f, ax = plt.subplots(ncols = 4, nrows = 1, figsize= (24, 5))

for i in range(len(target_amiata.columns)):
    sns.distplot(target_amiata[target_amiata.columns[i]], ax=ax[i])


print('### AMIATA_TARGET ###')

##### Lupa

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_lupa = lupa[[target for target in lupa.columns if target.startswith('Flow')]]
target_lupa_list = target_lupa.columns.tolist()
print('Number of Target : ' + str(len(target_lupa_list)))
print(target_lupa_list)

print()

# Feature variables
feature_lupa = lupa[[feature for feature in lupa.columns if feature not in target_lupa]]
feature_lupa_list = feature_lupa.columns.tolist()
print('Number of Features : ' + str(len(feature_lupa_list)))
print(feature_lupa_list)

In [ ]:
## HISTOGRAM (LUPA FEATURE VARIABLE)

f, ax = plt.subplots(ncols = 1, nrows = 1, figsize= (12, 5))
sns.distplot(feature_lupa[feature_lupa.columns[1]], ax=ax)

print('### LUPA_FEATURE ###')

In [ ]:
## HISTOGRAM (LUPA TARGET VARIABLE)

f, ax = plt.subplots(ncols = 1, nrows = 1, figsize= (12, 5))
sns.distplot(target_lupa[target_lupa.columns[0]], ax=ax)

print('### LUPA_TARGET ###')

##### Madonna di Canneto

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_madonna = madonna[[target for target in madonna.columns if target.startswith('Flow')]]
target_madonna_list = target_madonna.columns.tolist()
print('Number of Target : ' + str(len(target_lupa_list)))
print(target_madonna_list)

print()

# Feature variables
feature_madonna = madonna[[feature for feature in madonna.columns if feature not in target_madonna]]
feature_madonna_list = feature_madonna.columns.tolist()
print('Number of Features : ' + str(len(feature_madonna_list)))
print(feature_madonna_list)

In [ ]:
## HISTOGRAM (MADONNA_DI_CANNETO FEATURE VARIABLES)

# Number of columns
ncols = 2
nrows = math.ceil((len(feature_madonna.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(feature_madonna[feature_madonna.columns[i+1]], ax=ax[i]) for i in range(0,len(feature_madonna.columns)-1)]

print('### MADONNA_DI_CANNETO FEATURES ###')

In [ ]:
## HISTOGRAM (MADONNA_DI_CANNETO TARGET VARIABLE)

f, ax = plt.subplots(ncols = 1, nrows = 1, figsize= (12, 5))
sns.distplot(target_madonna[target_madonna.columns[0]], ax=ax)

print('### MADONNA_DI_CANNETO TARGET ###')

#### Plotting

##### Amiata

In [ ]:
columns = feature_amiata.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(amiata['Date'], amiata[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
columns = target_amiata.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(amiata['Date'], amiata[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

##### Lupa

In [ ]:
fig, axs = plt.subplots(nrows = 1, figsize= (10, 5))
axs.plot(lupa['Date'],lupa['Rainfall_Terni'])
axs.set_xlabel('Date')
axs.set_ylabel('Rainfall_Terni', fontsize = 15)
axs.grid()

In [ ]:
fig, axs = plt.subplots(nrows = 1, figsize= (10, 5))
axs.plot(lupa['Date'],lupa['Flow_Rate_Lupa'])
axs.set_xlabel('Date')
axs.set_ylabel('Flow_Rate_Lupa', fontsize = 15)
axs.grid()

In [ ]:
target_lupa_list

##### Madonna di Canneto

In [ ]:
columns = feature_madonna.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(madonna['Date'], madonna[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
fig, axs = plt.subplots(nrows = 1, figsize= (10, 5))
axs.plot(madonna['Date'],madonna['Flow_Rate_Madonna_di_Canneto'])
axs.set_xlabel('Date')
axs.set_ylabel('Flow_Rate_Madonna_di_Canneto', fontsize = 15)
axs.grid()

### RIVER

#### View Variables

In [ ]:
# List of Dataframes columns

columns_arno = arno.columns.tolist()
print(columns_arno)

##### Arno
 Arno is the second largest river in peninsular Italy and the main waterway in Tuscany and it has a relatively torrential regime, due to the nature of the surrounding soils (marl and impermeable clays). 
 Arno is the main source of water supply of Florence-Prato-Pistoia.

In [ ]:
print('Number of variables : ' + str(len(columns_arno)))

print('type of variables : ')
print(arno.dtypes)

In [ ]:
arno.describe()

#### Histogram

##### Arno

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_arno = arno[[target for target in arno.columns if target.startswith('Hydrometry')]]
target_arno_list = target_arno.columns.tolist()
print('Number of Target : ' + str(len(target_arno_list)))
print(target_arno_list)

print()

# Feature variables
feature_arno = arno[[feature for feature in arno.columns if feature not in target_arno]]
feature_arno_list = feature_arno.columns.tolist()
print('Number of Features : ' + str(len(feature_arno_list)))
print(feature_arno_list)

In [ ]:
## HISTOGRAM (ARNO FEATURE VARIABLES)

# Number of columns
ncols = 3
nrows = math.ceil((len(feature_arno.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(feature_arno[feature_arno.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_arno.columns)-1)]

print('### ARNO_FEATURES ###')

In [ ]:
## HISTOGRAM (ARNO TARGET VARIABLE)

f, ax = plt.subplots(ncols = 1, nrows = 1, figsize= (12, 5))
sns.distplot(target_arno[target_arno.columns[0]], ax=ax)

print('### ARNO TARGET ###')

#### Plotting

##### Arno

In [ ]:
columns = feature_arno.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(arno['Date'], arno[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
fig, axs = plt.subplots(nrows = 1, figsize= (10, 5))
axs.plot(arno['Date'],arno['Hydrometry_Nave_di_Rosano'])
axs.set_xlabel('Date')
axs.set_ylabel('Hydrometry_Nave_di_Rosano', fontsize = 15)
axs.grid()

### LAKE

#### View Variables

In [ ]:
# List of Dataframes columns

columns_bilancino = bilancino.columns.tolist()
print(columns_bilancino)

##### Bilancino

Bilancino lake is an artificial lake located in the municipality of Barberino di Mugello (about 50 km from Florence). It is used to refill the Arno river during the summer months. Indeed, during the winter months, the lake is filled up and then, during the summer months, the water of the lake is poured into the Arno river.

- Lake Level(m)

- Flow Rate : cubic meters per seconds (mc/s)

In [ ]:
print('Number of variables : ' + str(len(columns_bilancino)))

print('type of variables : ')
print(bilancino.dtypes)

In [ ]:
bilancino.describe()

#### Histogram

##### Bilancino

In [ ]:
## DIVISION DATAFRAME

# Target variables
target_bilancino = bilancino[[target for target in bilancino.columns if target.startswith('Lake') or target.startswith('Flow')]]
target_bilancino_list = target_bilancino.columns.tolist()
print('Number of Target : ' + str(len(target_bilancino_list)))
print(target_bilancino_list)

print()

# Feature variables
feature_bilancino = bilancino[[feature for feature in bilancino.columns if feature not in target_bilancino]]
feature_bilancino_list = feature_bilancino.columns.tolist()
print('Number of Features : ' + str(len(feature_bilancino_list)))
print(feature_bilancino_list)

In [ ]:
## HISTOGRAM (BILANCINO FEATURE VARIABLES)

# Number of columns
ncols = 3
nrows = math.ceil((len(feature_bilancino.columns)-1) / ncols)
f, ax = plt.subplots(ncols = ncols, nrows = nrows, figsize= (24, 5*nrows))
grid = []

for i in range(nrows):
    for j in range(ncols):
        grid.append([i,j])
[sns.distplot(feature_bilancino[feature_bilancino.columns[i+1]], ax=ax[grid[i][0], grid[i][1]]) for i in range(0,len(feature_bilancino.columns)-1)]

print('### BILANCINO_FEATURES ###')

In [ ]:
## HISTOGRAM (BILANCINO TARGET VARIABLES)

f, ax = plt.subplots(ncols = 2, nrows = 1, figsize= (25, 5))
grid = []

for i in range(len(target_bilancino.columns)):
    sns.distplot(feature_bilancino[feature_bilancino.columns[i+1]], ax=ax[i])

print('### BILANCINO TARGET ###')

#### Plotting

##### Bilancino

In [ ]:
columns = feature_bilancino.columns.to_list()
# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns) -1, figsize= (10, 5 * len(columns)))

for var in range(1,len(columns)):
    axs[var-1].plot(bilancino['Date'], bilancino[columns[var]])
    
    axs[var-1].set_xlabel('Date')
    axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
    axs[var-1].grid()

In [ ]:
columns = target_bilancino.columns.to_list()

# Plotting the time series for every variable
fig, axs = plt.subplots(nrows = len(columns), figsize= (10, 5 * len(columns)))

for var in range(0,len(columns)):
    axs[var].plot(bilancino['Date'], bilancino[columns[var]])
    
    axs[var].set_xlabel('Date')
    axs[var].set_ylabel(columns[var], fontsize = 15)
    
    axs[var].grid()

## Data on geographical view
With work of [EDA:Quenching the Thirst for Insights](https://www.kaggle.com/iamleonie/eda-quenching-the-thirst-for-insights), we will gather the coordinates for each location and than we could have an insight about relation amongs the locations.

And there is relation ship between the `lake Bilancino` and the `river Arno`(see [Wikipedia: Lago di Bilancino](https://en.wikipedia.org/wiki/Lago_di_Bilancino), [Wikipedia: Sieve (river)]( https://en.wikipedia.org/wiki/Sieve_(river))) Because  Lake Bilancino is an artificial lake made with a dam on the Sieve river. And the Sieve River is the most important tributary of the Arno river.

Water spring Lupa and Madonna have one location but river Arno(15),aquifer Auser(12), aquifer Luco(10) and the rest has the former. 

We will use this location information for missing data et feature engineering.

* Need to install module `folium` and `geopy`

In [ ]:
locations = {}

locations['Settefrati'] = {'lat' : 41.669624, 'lon' : 13.850011 }
locations['Velletri'] = {'lat' : 41.6867015, 'lon' : 12.7770433 }
locations['Petrignano'] = {'lat' : 43.1029282, 'lon' : 12.5237369 }
locations['Piaggione'] = {'lat' : 43.936794, 'lon' : 10.5040929 }
locations['S_Fiora'] = {'lat' : 42.854, 'lon' : 11.556 }
locations['Abbadia_S_Salvatore'] = {'lat' : 42.8809724, 'lon' : 11.6724203 }
locations['Vetta_Amiata'] = {'lat' : 42.8908958, 'lon' : 11.6264863 }
locations['Castel_del_Piano'] = {'lat' : 42.8932352, 'lon' : 11.5383804 }
locations['Terni'] = {'lat' : 42.6537515, 'lon' : 12.43981163 }
locations['Bastia_Umbra'] = {'lat' : 43.0677554, 'lon' : 12.5495816  }
locations['S_Savino'] = {'lat' : 43.339, 'lon' : 11.742 }
locations['Monteroni_Arbia_Biena'] = {'lat' : 43.228279, 'lon' : 11.4021433 }
locations['Monticiano_la_Pineta'] = {'lat' : 43.1335066 , 'lon' : 11.2408464 }
locations['Montalcinello'] = {'lat' : 43.1978783, 'lon' : 11.0787906 }
locations['Sovicille'] = {'lat' : 43.2806018, 'lon' : 11.2281756 }
locations['Simignano'] = {'lat' : 43.2921965, 'lon' : 11.1680079 }
locations['Mensano'] = {'lat' : 43.3009594 , 'lon' : 11.0548528 }
locations['Siena_Poggio_al_Vento'] = {'lat' : 43.1399762, 'lon' : 11.3832092 }
locations['Scorgiano'] = {'lat' : 43.3521445 , 'lon' : 11.15867 }
locations['Ponte_Orgia'] = {'lat' : 43.2074581 , 'lon' : 11.2504416 }
locations['Pentolina'] = {'lat' : 43.1968029, 'lon' : 11.1754672 }
locations['Montevarchi'] = {'lat' : 43.5234999, 'lon' : 11.5675911 }
locations['Incisa'] = {'lat' : 43.6558723, 'lon' : 11.4526838 }
locations['Camaldoli'] = {'lat' : 43.7943293, 'lon' : 11.8199481 }
locations['Bibbiena'] = {'lat' : 43.6955475, 'lon' : 11.817341 }
locations['Stia'] = {'lat' : 43.801537, 'lon' : 11.7067347 }
locations['Laterina'] = {'lat' : 43.5081823, 'lon' : 11.7102588 }
locations['Monteporzio'] = {'lat' : 41.817251, 'lon' : 12.7050839 }
locations['Pontetetto'] = {'lat' : 43.8226294, 'lon' : 10.4940843 }
locations['Ponte_a_Moriano'] = {'lat' : 43.9083609 , 'lon' : 10.5342488 }
locations['Calavorno'] = {'lat' : 44.0217216, 'lon' : 10.5297323 }
locations['Borgo_a_Mozzano'] = {'lat' : 43.978948, 'lon' : 10.545703  }
locations['Gallicano'] = {'lat' : 44.0606512, 'lon' : 10.435668  }
locations['Tereglio_Coreglia_Antelminelli'] = {'lat' : 44.0550548 , 'lon' : 10.5623594 }
locations['Lucca_Orto_Botanico'] = {'lat' : 43.84149865, 'lon' : 10.51169066 }
locations['Orentano'] = {'lat' : 43.7796506, 'lon' : 10.6583892 }
locations['Fabbriche_di_Vallico'] = {'lat' : 43.997647, 'lon' : 10.4279  }
locations['Monte_Serra'] = {'lat' : 43.750833, 'lon' : 10.555278 }
locations['Mangona'] = {'lat' : 44.0496863, 'lon' : 11.1958797 }
locations['Le_Croci'] = {'lat' : 44.0360503, 'lon' : 11.2675661 }
locations['Cavallina'] = {'lat' : 43.9833515, 'lon' : 11.2323312 }
locations['S_Agata'] = {'lat' : 43.9438247, 'lon' : 11.3089835 }
locations['Firenze'] = {'lat' : 43.7698712, 'lon' : 11.2555757 }
locations['S_Piero'] = {'lat' : 43.9637372, 'lon' : 11.3182991 }
locations['Vernio'] = {'lat' : 44.0440508 , 'lon' : 11.1498804  }
locations['Consuma'] = {'lat' : 43.784, 'lon' : 11.585 }
locations['Croce_Arcana']  = {'lat' : 44.1323056, 'lon' : 10.7689152 }
locations['Laghetto_Verde']  = {'lat' :   42.883, 'lon' : 11.662  }

locations_df = pd.DataFrame(columns=['city', 'lat', 'lon'] )

def get_location_coordinates(df, column_type, cluster, target_df):
    for location in df.columns[df.columns.str.startswith(column_type)]:
        location = location.split(column_type)[1]

        loc_dict = {}
        loc_dict['city'] = location
        loc_dict['cluster'] = cluster
        loc_dict['lat'] = locations[location]['lat']
        loc_dict['lon'] = locations[location]['lon']

        target_df = target_df.append(loc_dict, ignore_index=True)

    return target_df

locations_df = get_location_coordinates(auser, 'Temperature_', 'auser', locations_df)
locations_df = get_location_coordinates(auser, 'Rainfall_', 'auser', locations_df)

locations_df = get_location_coordinates(doganella, 'Temperature_', 'doganella', locations_df)
locations_df = get_location_coordinates(doganella, 'Rainfall_', 'doganella', locations_df)

locations_df = get_location_coordinates(luco, 'Temperature_', 'luco', locations_df)
locations_df = get_location_coordinates(luco, 'Rainfall_', 'luco', locations_df)

locations_df = get_location_coordinates(petrignano, 'Temperature_', 'petrignano', locations_df)
locations_df = get_location_coordinates(petrignano, 'Rainfall_', 'petrignano', locations_df)

locations_df = get_location_coordinates(bilancino, 'Temperature_', 'bilancino', locations_df)
locations_df = get_location_coordinates(bilancino, 'Rainfall_', 'bilancino', locations_df)

locations_df = get_location_coordinates(arno, 'Temperature_', 'arno', locations_df)
locations_df = get_location_coordinates(arno, 'Rainfall_', 'arno', locations_df)

locations_df = get_location_coordinates(amiata, 'Temperature_', 'amiata', locations_df)
locations_df = get_location_coordinates(amiata, 'Rainfall_', 'amiata', locations_df)

locations_df = get_location_coordinates(lupa, 'Temperature_', 'lupa', locations_df)
locations_df = get_location_coordinates(lupa, 'Rainfall_', 'lupa', locations_df)

locations_df = get_location_coordinates(madonna, 'Temperature_', 'madonna', locations_df)
locations_df = get_location_coordinates(madonna, 'Rainfall_', 'madonna', locations_df)

# Drop duplicates
locations_df = locations_df.sort_values(by='city').drop_duplicates().reset_index(drop=True)

# Label Encode cluster feature for visualization puposes
le = preprocessing.LabelEncoder()
le.fit(locations_df.cluster)
locations_df['cluster_enc'] = le.transform(locations_df.cluster)

m = folium.Map(location=[42.6, 12.4], tiles='cartodbpositron',zoom_start=7)

colors = ['purple','lightred','green', 'lightblue', 'red', 'blue', 'darkblue','lightgreen', 'orange',  'darkgreen', 'beige',  'pink', 'darkred', 'darkpurple', 'cadetblue',]

geolocator = Nominatim(user_agent='myapplication')
for i in locations_df.index:
    folium.Marker([locations_df.iloc[i].lat, 
                  locations_df.iloc[i].lon],
                  popup=locations_df.iloc[i].city, 
                  icon=folium.Icon(color=colors[locations_df.iloc[i].cluster_enc])).add_to(m)

m

# DATA PROCESSING

## HANDLING MISSING DATA

### AQUIFER

##### Auser

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
sns.heatmap(auser.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
date = '2014-01-01'
bool_filter = pd.to_datetime(auser['Date'], errors='coerce').ge(date)
auser = auser[bool_filter].reset_index(drop=True)
auser.head()

In [ ]:
auser['Temperature_Ponte_a_Moriano'] = np.where((auser['Temperature_Ponte_a_Moriano'] == 0),
                                                auser['Temperature_Lucca_Orto_Botanico'], auser['Temperature_Ponte_a_Moriano'])

In [ ]:
tmp = auser[auser['Date'] >= '2019-06-22']
tmp = tmp[tmp['Date'] <= '2019-06-26'].reset_index(drop=True).reset_index(drop=True)
tmp['Temperature_Monte_Serra'] = np.where((tmp['Temperature_Monte_Serra'] == 0), np.nan, tmp['Temperature_Monte_Serra'])

for i in range(len(tmp)):
  auser['Temperature_Monte_Serra'] = np.where((auser['Date'] == tmp['Date'][i]), tmp['Temperature_Monte_Serra'][i], auser['Temperature_Monte_Serra'])

In [ ]:
col = ['Depth_to_Groundwater_LT2',
       'Depth_to_Groundwater_SAL',
       'Depth_to_Groundwater_CoS']
columns = auser.loc[:, col[:]].columns.to_list()
for column in columns:
  print(column)
  auser[f'{column}'] = np.where((auser[f'{column}'] == 0), np.nan, auser[f'{column}'])

In [ ]:

col = auser.columns.to_list()
columns = auser.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  auser[f'{column}'] = auser[f'{column}'].interpolate()

In [ ]:
auser.Depth_to_Groundwater_CoS.isnull().sum()

In [ ]:
auser = auser[auser.Depth_to_Groundwater_CoS.notna()].reset_index(drop=True)
auser.info()

In [ ]:
auser.isnull().sum()

In [ ]:
# columns = auser.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(auser['Date'], auser[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

##### Doganella

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,5))
sns.heatmap(doganella.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_fontsize(14) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
temperature_cols=[]
for column in doganella.columns:
    if 'Temperature' in column:
        temperature_cols.append(column)
temperature_cols

In [ ]:
'''
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import gc
import lightgbm as lgb
'''

def get_time_features(df):
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['day_in_year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['week_in_year'] = pd.DatetimeIndex(df['Date']).weekofyear
    return df

original = doganella.copy()
tmp_original = get_time_features(original)
tmp_original.month = tmp_original.month.astype(int)
tmp_original.year = tmp_original.year.astype(int)

params = {'num_leaves': 24,
          'objective': 'regression_l1',
          'max_depth': 8,
          'learning_rate': 0.01,
          "metric": 'mae',
          "verbosity": -1,
          'verbose': -1,
          'seed' : 42
         }

def predict_missing_temperature(target):
    test_df = tmp_original.copy()
    train_df = tmp_original[tmp_original[target].notna()]

    features = [c for c in (list(temperature_cols) + list(['month', 'day_in_year', 'week_in_year'])) if c != target]

    X = train_df[features]
    y = train_df[[target]]
    X_test = test_df[features]

    y_preds = np.zeros(X_test.shape[0])

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dtrain = lgb.Dataset(X_train, y_train, params= {'verbose': -1})
    dvalid = lgb.Dataset(X_valid, y_valid, params= {'verbose': -1})

    # For analysis set 'verbose_eval' to 200, false
    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=False,  early_stopping_rounds=100)

    y_pred_valid = clf.predict(X_valid)
    y_preds = clf.predict(X_test)
    
    f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 4))
    ax.set_title(f'Rolling Mean Temperatures for {target} \n(Rolling Window: 28 Days, MAE: {mean_absolute_error(y_valid.rolling(28).mean()[28:], pd.Series(y_pred_valid).rolling(28).mean()[28:])})', fontsize=16)
    old = tmp_original[target].copy().replace({np.nan : np.inf})
    tmp_original[target] = np.where(tmp_original[target].isna(), pd.Series(y_preds), tmp_original[target])
    sns.lineplot(x=original.Date, y=tmp_original[target].rolling(28).mean(), label='Imputed', color='darkorange')
    sns.lineplot(x=original.Date, y=old.rolling(28).mean(), label='Original', color='dodgerblue')
    ax.set_xlabel('Date', fontsize=14)
#     ax.set_xlim([date(2010, 1, 1), date(2020, 6, 30)])
    plt.show()

    # return pd.Series(y_preds)
    return tmp_original[target]

# Sort temperature columns according to amount of missing values.
# Fill NaN values for features with least missing values first
sorted_temperature_cols = tmp_original[temperature_cols].isna().sum(axis=0).sort_values().index

example_col = 'Temperature_Monteporzio'
doganella['Temperature_Monteporzio'] = predict_missing_temperature(example_col)

example_col2 = 'Temperature_Velletri'
doganella['Temperature_Velletri'] = predict_missing_temperature(example_col2)

In [ ]:
doganella = doganella[doganella.Date > '2016-10-06'].reset_index(drop=True)
doganella.info()

In [ ]:
col = ['Volume_Pozzo_1', 'Volume_Pozzo_2',
       'Volume_Pozzo_3', 'Volume_Pozzo_4', 'Volume_Pozzo_5+6',
       'Volume_Pozzo_7', 'Volume_Pozzo_8', 'Volume_Pozzo_9']
columns = doganella.loc[:, col[:]].columns.to_list()
for column in columns:
  print(column)
  doganella[f'{column}'] = np.where((doganella[f'{column}'] == 0), np.nan, doganella[f'{column}'])

In [ ]:
col = doganella.columns.to_list()
columns = doganella.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  doganella[f'{column}'] = doganella[f'{column}'].interpolate()

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,5))
sns.heatmap(doganella.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_fontsize(14) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
doganella = doganella[doganella.Depth_to_Groundwater_Pozzo_4.notna()].reset_index(drop=True)
doganella.info()

In [ ]:
doganella.isnull().sum()

In [ ]:
# columns = doganella.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(doganella['Date'], doganella[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

##### Luco

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))
sns.heatmap(luco.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_fontsize(14) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
luco = luco[luco.Rainfall_Siena_Poggio_al_Vento.notna()].reset_index(drop=True)
luco.info()

In [ ]:
luco = luco.drop(['Depth_to_Groundwater_Podere_Casetta'], axis=1)
luco.columns

In [ ]:
col = luco.columns.to_list()
columns = luco.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  luco[f'{column}'] = luco[f'{column}'].interpolate()

In [ ]:
luco.isnull().sum()

In [ ]:
# columns = luco.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(luco['Date'], luco[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

##### Petrignano

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(16,5))
sns.heatmap(petrignano.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_fontsize(14) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
petrignano = petrignano[petrignano.Rainfall_Bastia_Umbra.notna()].reset_index(drop=True)
petrignano.info()

In [ ]:
col = ['Volume_C10_Petrignano',
       'Hydrometry_Fiume_Chiascio_Petrignano']
columns = petrignano.loc[:, col[:]].columns.to_list()
for column in columns:
  print(column)
  petrignano[f'{column}'] = np.where((petrignano[f'{column}'] == 0), np.nan, petrignano[f'{column}'])

In [ ]:
col = petrignano.columns.to_list()
columns = petrignano.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  petrignano[f'{column}'] = petrignano[f'{column}'].interpolate()

In [ ]:
petrignano.isnull().sum()

In [ ]:
# columns = petrignano.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(petrignano['Date'], petrignano[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

### WATER SPRING

##### Amiata

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,8))
sns.heatmap(amiata.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
amiata = amiata[amiata.Flow_Rate_Galleria_Alta.notna()].reset_index(drop=True)
amiata.info()

In [ ]:
amiata = amiata[amiata.Rainfall_S_Fiora.notna()].reset_index(drop=True)
amiata.info()

In [ ]:
col = ['Flow_Rate_Bugnano', 'Flow_Rate_Arbure', 'Flow_Rate_Ermicciolo']
columns = amiata.loc[:, col[:]].columns.to_list()
for column in columns:
  print(column)
  amiata[f'{column}'] = np.where((amiata[f'{column}'] == 0), np.nan, amiata[f'{column}'])

In [ ]:
col = amiata.columns.to_list()
columns = amiata.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  amiata[f'{column}'] = amiata[f'{column}'].interpolate()

In [ ]:
amiata = amiata[amiata.Rainfall_Abbadia_S_Salvatore.notna()].reset_index(drop=True)
amiata.info()

In [ ]:
amiata.isnull().sum()

In [ ]:
# columns = amiata.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(amiata['Date'], amiata[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

##### Lupa

##### Madonna di Canneto

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,8))
sns.heatmap(madonna.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
madonna = madonna[madonna['Date'] < '2019-01-01'].reset_index(drop=True)
madonna.isnull().sum()

In [ ]:
madonna = madonna[madonna['Date'] >= '2015-03-13'].reset_index(drop=True)
madonna.head()

In [ ]:
col = madonna.columns.to_list()
columns = madonna.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  madonna[f'{column}'] = madonna[f'{column}'].interpolate()

In [ ]:
madonna.isnull().sum()

In [ ]:
# columns = madonna.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(madonna['Date'], madonna[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

### RIVER

##### Arno

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
sns.heatmap(arno.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
columns = ['Date', 'Rainfall_Le_Croci', 'Rainfall_Cavallina', 'Rainfall_S_Agata',
       'Rainfall_Mangona', 'Rainfall_S_Piero', 'Temperature_Firenze',
       'Hydrometry_Nave_di_Rosano']
arno = arno[columns]
arno.info()

In [ ]:
temperature_cols=[]
for column in arno.columns:
    if 'Temperature' in column:
        temperature_cols.append(column)
temperature_cols

In [ ]:
def get_time_features(df):
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['day_in_year'] = pd.DatetimeIndex(df['Date']).dayofyear
    df['week_in_year'] = pd.DatetimeIndex(df['Date']).weekofyear
    return df

original = arno.copy()
tmp_original = get_time_features(original)
tmp_original.month = tmp_original.month.astype(int)
tmp_original.year = tmp_original.year.astype(int)

params = {'num_leaves': 24,
          'objective': 'regression_l1',
          'max_depth': 8,
          'learning_rate': 0.01,
          "metric": 'mae',
          "verbosity": -1,
          'verbose': -1,
          'seed' : 42
         }

def predict_missing_temperature(target):
    test_df = tmp_original.copy()
    train_df = tmp_original[tmp_original[target].notna()]

    features = [c for c in (list(temperature_cols) + list(['month', 'day_in_year', 'week_in_year'])) if c != target]

    X = train_df[features]
    y = train_df[[target]]
    X_test = test_df[features]

    y_preds = np.zeros(X_test.shape[0])

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dtrain = lgb.Dataset(X_train, y_train, params= {'verbose': -1})
    dvalid = lgb.Dataset(X_valid, y_valid, params= {'verbose': -1})

    # For analysis set 'verbose_eval' to 200, false
    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=False,  early_stopping_rounds=100)

    y_pred_valid = clf.predict(X_valid)
    y_preds = clf.predict(X_test)
    
    f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 4))
    ax.set_title(f'Rolling Mean Temperatures for {target} \n(Rolling Window: 28 Days, MAE: {mean_absolute_error(y_valid.rolling(28).mean()[28:], pd.Series(y_pred_valid).rolling(28).mean()[28:])})', fontsize=16)
    old = tmp_original[target].copy().replace({np.nan : np.inf})
    tmp_original[target] = np.where(tmp_original[target].isna(), pd.Series(y_preds), tmp_original[target])
    sns.lineplot(x=original.Date, y=tmp_original[target].rolling(28).mean(), label='Imputed', color='darkorange')
    sns.lineplot(x=original.Date, y=old.rolling(28).mean(), label='Original', color='dodgerblue')
    ax.set_xlabel('Date', fontsize=14)
    # ax.set_xlim([date(2014, 1, 1), date(2020, 6, 30)])
    plt.show()

    # return pd.Series(y_preds)
    return tmp_original[target]

# Sort temperature columns according to amount of missing values.
# Fill NaN values for features with least missing values first
sorted_temperature_cols = tmp_original[temperature_cols].isna().sum(axis=0).sort_values().index

example_col = 'Temperature_Firenze'
arno['Temperature_Firenze'] = predict_missing_temperature(example_col)

In [ ]:
col = ['Hydrometry_Nave_di_Rosano']
columns = arno.loc[:, col[:]].columns.to_list()
for column in columns:
  print(column)
  arno[f'{column}'] = np.where((arno[f'{column}'] == 0), np.nan, arno[f'{column}'])

In [ ]:
col = arno.columns.to_list()
columns = arno.loc[:, col[1:]].columns.to_list()
for column in columns:
  print(column)
  arno[f'{column}'] = arno[f'{column}'].interpolate()

In [ ]:
arno = arno[arno.Rainfall_Le_Croci.notna()].reset_index(drop=True)
arno.info()

In [ ]:
arno.isnull().sum()

In [ ]:
# columns = arno.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(arno['Date'], arno[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

### LAKE

##### Bilancino

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))
sns.heatmap(bilancino.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(14)
plt.show()

In [ ]:
bilancino = bilancino[bilancino.Temperature_Le_Croci.notna()].reset_index(drop=True)
bilancino.info()

In [ ]:
bilancino.isnull().sum()

In [ ]:
# columns = bilancino.columns.to_list()
# # Plotting the time series for every variable
# fig, axs = plt.subplots(nrows = len(columns)-1, figsize= (10, 5 * len(columns)))


# for var in range(1,len(columns)):
#     axs[var-1].plot(bilancino['Date'], bilancino[columns[var]])
    
#     axs[var-1].set_xlabel('Date')
#     axs[var-1].set_ylabel(columns[var], fontsize = 15)
    
#     axs[var-1].grid()

## FEATURE ENGINEERING

### AQUIFER

##### Auser

###### Check Rainfall correlation

###### - Extract Rainfall values

In [ ]:
Auser_Rainfall = auser.iloc[:,0:11]
Auser_Rainfall.head()

In [ ]:
Auser_Rainfall.set_index("Date",inplace=True)

In [ ]:
Auser_Rainfall_Relation=Auser_Rainfall.corr()
Auser_Rainfall_Relation.head()

It can be sure that the corrolation among region in Rainfall section.
- If it rains in one area, it is likely to rain in other area even if the amount can be different.

In [ ]:
sns.heatmap(Auser_Rainfall_Relation,annot=True)
Auser_Rainfall.resample(rule='A').mean()['2015']



###### check the corrolation by among the Depth_to_ground

In [ ]:
Auser_ground = auser.iloc[:,11:16]
Auser_ground_relation = Auser_ground.corr()
Auser_ground_relation

In [ ]:
sns.heatmap(Auser_ground_relation,annot=True)


###### check the corrolation by Temperature

In [ ]:
Temperature=auser[['Date','Temperature_Orentano','Temperature_Monte_Serra','Temperature_Ponte_a_Moriano','Temperature_Lucca_Orto_Botanico']]
Temperature


In [ ]:
Temperature_relation = Temperature.corr()
sns.heatmap(Temperature_relation,annot=True)



###### conclusion
   - Each region's climate is correlated except one.
   - The rainfall in each region shows a correlation except one.
   - The value of each Depth_to_underground is correlated except one.
   - Then climate is similar in all but one area which mean that we can assume 3 of them are in similar region.



### WATER SPRING

##### Amiata

### RIVER

##### Arno

In [ ]:
plot_corr_hitmap(arno, 'River Arno')

###### Check Rainfall correlation
There is two correlated groups of rainfall. with geograpical analysis, we have found why there are two rainfall groups. Vernio, Mangona, Cavallina, Le_Croci, S_Agata and S_Piero are to the north of Firenze, near to Lake Bilancino, these precipitations reach the River Arno through the River Sieve. The other points are located to the southeast of Firenze and fall directly into the Arno riverbed.

We chose Rainfall_S_Piero to represent precipitation in the north and Rainfall_Incisa for precipitation in the south.

In [ ]:
bilancino.info()

In [ ]:
arno.info()

In [ ]:
tmp = pd.merge(arno, bilancino, left_on='Date', right_on='Date', how='left')
columns = ['Date', 'Rainfall_Le_Croci_x', 'Rainfall_Cavallina_x', 'Rainfall_S_Agata_x',
       'Rainfall_Mangona_x', 'Rainfall_S_Piero_x', 'Temperature_Firenze',
       'Hydrometry_Nave_di_Rosano', 'Lake_Level', 'Flow_Rate']
arno = tmp[columns]

new_columns = ['Date', 'Rainfall_Le_Croci', 'Rainfall_Cavallina', 'Rainfall_S_Agata',
       'Rainfall_Mangona', 'Rainfall_S_Piero', 'Temperature_Firenze',
       'Hydrometry_Nave_di_Rosano', 'Lake_Level', 'Flow_Rate']
arno.columns = new_columns
arno.columns

In [ ]:
arno.info()

In [ ]:
arno = arno[arno.Lake_Level.notna()].reset_index(drop=True)
arno.info()

### LAKE

#### Bilancino
 Bilancino lake is an artificial lake located in the municipality of Barberino di Mugello (about 50 km from Florence). It is used to refill the Arno river during the summer months. Indeed, during the winter months, the lake is filled up and then, during the summer months, the water of the lake is poured into the Arno river.

In [ ]:
plot_corr_hitmap(bilancino, 'Lake Bilancino')

###### Check Rainfall correlation
In the case of Lake Bilancino, we select `the Rainfall_S_Piero` which has a strong correlation with the other precipitation variables.

# PREDICTIVE MODEL

## Prophet

### ACQUIFER

In [ ]:
auser.info()

In [ ]:
# As we can see, downsample to weekly could smooth the data and hgelp with analysis
columns = auser.columns.to_list()
downsample = auser[columns].resample('7D', on='Date').mean().reset_index(drop=False)
auser_ds = downsample.copy()

In [ ]:
auser_ds.info()

In [ ]:
auser_ds = auser_ds.drop(['Depth_to_Groundwater_LT2', 'Depth_to_Groundwater_PAG', 'Depth_to_Groundwater_CoS', 'Depth_to_Groundwater_DIEC'], axis=1)
auser_ds.columns

In [ ]:
auser_ds.info()

In [ ]:
target_column=[]
for column in auser_ds.columns:
    if 'Depth' in column:
        target_column.append(column)
target_column

In [ ]:
feature_columns = [feature for feature in auser_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(auser_ds))

multivariate_df = auser_ds[['Date'] + target_column + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Depth to Groundwater', fontsize=14)

plt.show()

In [ ]:
target_mean = abs(y_valid['y'].mean())
target_mean

In [ ]:
# Relative MAE
score_mae/target_mean

In [ ]:
# Relative RMSE
score_rmse/target_mean

### WATER SPRING

In [ ]:
amiata.info()

In [ ]:
# As we can see, downsample to weekly could smooth the data and hgelp with analysis
columns = amiata.columns.to_list()
downsample = amiata[columns].resample('7D', on='Date').mean().reset_index(drop=False)
amiata_ds = downsample.copy()

In [ ]:
amiata_ds.info()

In [ ]:
target_column=[]
for column in amiata_ds.columns:
    if 'Flow_Rate' in column:
        target_column.append(column)
target_column

In [ ]:
# CASE : 'Flow_Rate_Bugnano'
feature_columns = [feature for feature in amiata_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(amiata_ds))

multivariate_df = amiata_ds[['Date'] + [target_column[0]] + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Flow Rate', fontsize=14)

plt.show()

In [ ]:
Bugnano_target_mean = abs(y_valid['y'].mean())
Bugnano_target_mean

In [ ]:
# Relative MAE
score_mae/Bugnano_target_mean

In [ ]:
# Relative RMSE
score_rmse/Bugnano_target_mean

In [ ]:
# CASE : 'Flow_Rate_Arbure'
feature_columns = [feature for feature in amiata_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(amiata_ds))

multivariate_df = amiata_ds[['Date'] + [target_column[1]] + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Flow Rate', fontsize=14)

plt.show()

In [ ]:
Arbure_target_mean = abs(y_valid['y'].mean())
Arbure_target_mean

In [ ]:
# Relative MAE
score_mae/Arbure_target_mean

In [ ]:
# Relative RMSE
score_rmse/Arbure_target_mean

In [ ]:
# CASE : 'Flow_Rate_Ermicciolo'
feature_columns = [feature for feature in amiata_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(amiata_ds))

multivariate_df = amiata_ds[['Date'] + [target_column[2]] + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Flow Rate', fontsize=14)

plt.show()

In [ ]:
Ermicciolo_target_mean = abs(y_valid['y'].mean())
Ermicciolo_target_mean

In [ ]:
# Relative MAE
score_mae/Ermicciolo_target_mean

In [ ]:
# Relative RMSE
score_rmse/Ermicciolo_target_mean

In [ ]:
# CASE : 'Flow_Rate_Galleria_Alta'
feature_columns = [feature for feature in amiata_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(amiata_ds))

multivariate_df = amiata_ds[['Date'] + [target_column[3]] + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Flow Rate', fontsize=14)

plt.show()

In [ ]:
Galleria_Alta_target_mean = abs(y_valid['y'].mean())
Galleria_Alta_target_mean

In [ ]:
# Relative MAE
score_mae/Galleria_Alta_target_mean

In [ ]:
# Relative RMSE
score_rmse/Galleria_Alta_target_mean

### RIVER

In [ ]:
arno.info()

In [ ]:
# As we can see, downsample to weekly could smooth the data and hgelp with analysis
columns = arno.columns.to_list()
downsample = arno[columns].resample('7D', on='Date').mean().reset_index(drop=False)
arno_ds = downsample.copy()

In [ ]:
arno_ds.info()

In [ ]:
target_column=[]
for column in arno_ds.columns:
  if 'Hydrometry' in column:
    target_column.append(column)
target_column

In [ ]:
feature_columns = [feature for feature in arno_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(arno_ds))

multivariate_df = arno_ds[['Date'] + target_column + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Lake Level', fontsize=14)

plt.show()

In [ ]:
target_mean = abs(y_valid['y'].mean())
target_mean

In [ ]:
# Relative MAE
score_mae/target_mean

In [ ]:
# Relative RMSE
score_rmse/target_mean

### LAKE

In [ ]:
bilancino.info()

In [ ]:
# As we can see, downsample to weekly could smooth the data and hgelp with analysis
columns = bilancino.columns.to_list()
downsample = bilancino[columns].resample('7D', on='Date').mean().reset_index(drop=False)
bilancino_ds = downsample.copy()

In [ ]:
bilancino_ds.info()

In [ ]:
target_column=[]
for column in bilancino_ds.columns:
  if 'Lake_Level' in column:
    target_column.append(column)
  if 'Flow_Rate' in column:
    target_column.append(column)
target_column

In [ ]:
# CASE : Lake_Level
feature_columns = [feature for feature in bilancino_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(bilancino_ds))

multivariate_df = bilancino_ds[['Date'] + [target_column[0]] + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]

col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Lake Level', fontsize=14)

plt.show()

In [ ]:
Level_target_mean = abs(y_valid['y'].mean())
Level_target_mean

In [ ]:
# Relative MAE
score_mae/Level_target_mean

In [ ]:
# Relative RMSE
score_rmse/Level_target_mean

In [ ]:
# CASE : Flow_Rate
feature_columns = [feature for feature in bilancino_ds.columns.to_list() if feature not in target_column ][1:]

train_size = int(0.85 * len(bilancino_ds))

multivariate_df = bilancino_ds[['Date'] + [target_column[1]] + feature_columns].copy()
multivariate_df.columns = ['ds', 'y'] + feature_columns

train = multivariate_df.iloc[:train_size, :]
col_tmp = [i for i in range(2, len(feature_columns)+2)]
col_tmp.append(0)
col_tmp.sort()
x_train, y_train = pd.DataFrame(multivariate_df.iloc[:train_size, col_tmp]), pd.DataFrame(multivariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(multivariate_df.iloc[train_size:, col_tmp]), pd.DataFrame(multivariate_df.iloc[train_size:, 1])

train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from fbprophet import Prophet


# Train the model
model = Prophet()
for f_col in feature_columns:
  print(f_col)
  model.add_regressor(f'{f_col}')

# Fit the model with train set
model.fit(train)

# Predict on valid set
y_pred = model.predict(x_valid)

# Calcuate metrics
score_mae = mean_absolute_error(y_valid, y_pred['yhat'])
score_rmse = math.sqrt(mean_squared_error(y_valid, y_pred['yhat']))

print('RMSE: {}'.format(score_rmse))

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

model.plot(y_pred, ax=ax)
sns.lineplot(x=x_valid['ds'], y=y_valid['y'], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Flow Rate', fontsize=14)

plt.show()

In [ ]:
Rate_target_mean = abs(y_valid['y'].mean())
Rate_target_mean

In [ ]:
# Relative MAE
score_mae/Rate_target_mean

In [ ]:
# Relative RMSE
score_rmse/Rate_target_mean

## LSTM

### ACQUIFER

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
pd.set_option('display.max_columns',100)
pd.set_option('display.max_row',1000)

In [ ]:
data = auser.copy()
data['Date']=pd.to_datetime(data['Date'],format ='%d/%m/%Y')
data.set_index("Date",inplace=True)
data=data.dropna()
data

In [ ]:
data = auser.copy()
data.set_index("Date",inplace=True)
data

##### DepthToGround
    - It takes time for rainfall to seep into the ground and reach the groundwater.
    - I set the day as 3days

In [ ]:
DepthToGroundColumns=[]
for column in data.columns:
    if 'Depth' in column:
        DepthToGroundColumns.append(column)
DepthToGroundColumns

In [ ]:
dayToUnderground = 3
dataShift = data.copy()
dataShift[DepthToGroundColumns] = dataShift[DepthToGroundColumns].shift(-dayToUnderground,freq='D')
dataShift.tail()

In [ ]:
dataSet = dataShift.dropna()
dataSet

In [ ]:
dataSetExtract = dataSet.drop(DepthToGroundColumns,axis=1)
dataSetExtract.head()

In [ ]:
dataGround = dataSet[DepthToGroundColumns]
dataGround
dataModified = pd.concat([dataGround,dataSetExtract],axis=1)
dataModified.iloc[:,:-5]

#### Data Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataSetScaler = scaler.fit_transform(dataModified)
print(dataSetScaler.shape)

In [ ]:
dataSetScaled = dataSetScaler

In [ ]:
from numpy import array

In [ ]:
n_steps_in, n_steps_out = 120 , 20
def split_dataSet(data,n_steps_in,n_steps_out):
    X,y = list(),list()
    for i in range(len(dataSetScaled)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(data):
            break
        data_x,data_y = data[i:end_ix, :],data[end_ix-1:out_end_ix,:]
        X.append(data_x)
        y.append(data_y)
    return np.array(X), np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X,y = split_dataSet(dataSetScaled,n_steps_in,n_steps_out)
print(X.shape)
print(y.shape)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

n_features = x_train.shape[2]
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(x_train, y_train, epochs=100,validation_data=(x_test,y_test) ,verbose=1)

In [ ]:
predicData = model.predict(x_test, verbose=1)
predictlist20days=[]
for i in range(len(predicData[0])):
    predictlist20days.append(predicData[0][i][0])
days120list=[]
for i in range(len(x_test[0])):
    days120list.append(x_test[0][i][0])

days120list20days=[]
for i in range(len(y_test[0])):
    days120list20days.append(y_test[0][i][0])
predictlist20days

In [ ]:
aAxis=np.arange(0,len(days120list))
bAxis=np.arange(len(days120list)-1,len(days120list)+19)
plt.plot(aAxis,days120list)
plt.plot(bAxis,days120list20days,color='cyan')
plt.plot(bAxis,predictlist20days,color='red')

### WATER SPRING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
pd.set_option('display.max_columns',100)
pd.set_option('display.max_row',1000)

In [ ]:
data = amiata.copy()
data['Date']=pd.to_datetime(data['Date'],format ='%d/%m/%Y')
data.set_index("Date",inplace=True)
data=data.dropna()
data

In [ ]:
DepthToGroundColumns=[]
for column in data.columns:
    if 'Hydro' in column:
        DepthToGroundColumns.append(column)
DepthToGroundColumns

In [ ]:
dayToUnderground = 0
dataShift = data.copy()
dataShift[DepthToGroundColumns] = dataShift[DepthToGroundColumns].shift(-dayToUnderground,freq='D')
dataShift.tail()

In [ ]:
dataSet = dataShift.dropna()
dataSet

In [ ]:
dataSetExtract = dataSet.drop(DepthToGroundColumns,axis=1)
dataSetExtract.head()

In [ ]:
dataGround = dataSet[DepthToGroundColumns]
dataGround
dataModified = pd.concat([dataGround,dataSetExtract],axis=1)
dataModified.iloc[:,:-5]

##### Data Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataSetScaler = scaler.fit_transform(dataModified)
print(dataSetScaler.shape)

In [ ]:
dataSetScaled = dataSetScaler

In [ ]:
from numpy import array

In [ ]:
n_steps_in, n_steps_out = 120 , 20
def split_dataSet(data,n_steps_in,n_steps_out):
    X,y = list(),list()
    for i in range(len(dataSetScaled)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(data):
            break
        data_x,data_y = data[i:end_ix, :],data[end_ix-1:out_end_ix,:]
        X.append(data_x)
        y.append(data_y)
    return np.array(X), np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X,y = split_dataSet(dataSetScaled,n_steps_in,n_steps_out)
print(X.shape)
print(y.shape)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

n_features = x_train.shape[2]
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(x_train, y_train, epochs=100,validation_data=(x_test,y_test) ,verbose=1)



In [ ]:
predicData = model.predict(x_test, verbose=1)
predictlist20days=[]
for i in range(len(predicData[0])):
    predictlist20days.append(predicData[0][i][-1])
days120list=[]
for i in range(len(x_test[0])):
    days120list.append(x_test[0][i][-1])

days120list20days=[]
for i in range(len(y_test[0])):
    days120list20days.append(y_test[0][i][-1])
predictlist20days



In [ ]:
aAxis=np.arange(0,len(days120list))
bAxis=np.arange(len(days120list)-1,len(days120list)+19)
plt.plot(aAxis,days120list)
plt.plot(bAxis,days120list20days,color='cyan')
plt.plot(bAxis,predictlist20days,color='red')

### RIVER

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
pd.set_option('display.max_columns',100)
pd.set_option('display.max_row',1000)

In [ ]:
data = arno.copy()
data['Date']=pd.to_datetime(data['Date'],format ='%d/%m/%Y')
data.set_index("Date",inplace=True)
data=data.dropna()
data

In [ ]:
DepthToGroundColumns=[]
for column in data.columns:
    if 'Hydro' in column:
        DepthToGroundColumns.append(column)
DepthToGroundColumns

In [ ]:
dayToUnderground = 0
dataShift = data.copy()
dataShift[DepthToGroundColumns] = dataShift[DepthToGroundColumns].shift(-dayToUnderground,freq='D')
dataShift.tail()

In [ ]:
dataSet = dataShift.dropna()
dataSet

In [ ]:
dataSetExtract = dataSet.drop(DepthToGroundColumns,axis=1)
dataSetExtract.head()

In [ ]:
dataGround = dataSet[DepthToGroundColumns]
dataGround
dataModified = pd.concat([dataGround,dataSetExtract],axis=1)
dataModified.iloc[:,:-5]

##### Data Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataSetScaler = scaler.fit_transform(dataModified)
print(dataSetScaler.shape)

In [ ]:
from numpy import array

In [ ]:
n_steps_in, n_steps_out = 120 , 20
def split_dataSet(data,n_steps_in,n_steps_out):
    X,y = list(),list()
    for i in range(len(dataSetScaled)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(data):
            break
        data_x,data_y = data[i:end_ix, :],data[end_ix-1:out_end_ix,:]
        X.append(data_x)
        y.append(data_y)
    return np.array(X), np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X,y = split_dataSet(dataSetScaled,n_steps_in,n_steps_out)
print(X.shape)
print(y.shape)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

n_features = x_train.shape[2]
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(x_train, y_train, epochs=5,batch_size=1,validation_data=(x_test,y_test) ,verbose=1)



In [ ]:
predicData = model.predict(x_test, verbose=1)
predictlist20days=[]
for i in range(len(predicData[0])):
    predictlist20days.append(predicData[0][i][0])
days120list=[]
for i in range(len(x_test[0])):
    days120list.append(x_test[0][i][0])

days120list20days=[]
for i in range(len(y_test[0])):
    days120list20days.append(y_test[0][i][0])
predictlist20days

In [ ]:
aAxis=np.arange(0,len(days120list))
bAxis=np.arange(len(days120list)-1,len(days120list)+19)
plt.plot(aAxis,days120list)
plt.plot(bAxis,days120list20days,color='cyan')
plt.plot(bAxis,predictlist20days,color='red')

### LAKE

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
pd.set_option('display.max_columns',100)
pd.set_option('display.max_row',1000)

In [ ]:
data = bilancino.copy()
data['Date']=pd.to_datetime(data['Date'],format ='%d/%m/%Y')
data.set_index("Date",inplace=True)
data=data.dropna()
data

In [ ]:
DepthToGroundColumns=[]
for column in data.columns:
    if 'Depth' in column:
        DepthToGroundColumns.append(column)
DepthToGroundColumns

In [ ]:
dayToUnderground = 0
dataShift = data.copy()
dataShift[DepthToGroundColumns] = dataShift[DepthToGroundColumns].shift(-dayToUnderground,freq='D')
dataShift.tail()

In [ ]:
dataSet = dataShift.dropna()
dataSet

In [ ]:
dataSetExtract = dataSet.drop(DepthToGroundColumns,axis=1)
dataSetExtract.head()

In [ ]:
dataGround = dataSet[DepthToGroundColumns]
dataGround
dataModified = pd.concat([dataGround,dataSetExtract],axis=1)
dataModified.iloc[:,:-5]

##### Data Standardization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataSetScaler = scaler.fit_transform(dataModified)
print(dataSetScaler.shape)

In [ ]:
dataSetScaled = dataSetScaler

In [ ]:
from numpy import array

In [ ]:
n_steps_in, n_steps_out = 120 , 20
def split_dataSet(data,n_steps_in,n_steps_out):
    X,y = list(),list()
    for i in range(len(dataSetScaled)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(data):
            break
        data_x,data_y = data[i:end_ix, :],data[end_ix-1:out_end_ix,:]
        X.append(data_x)
        y.append(data_y)
    return np.array(X), np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X,y = split_dataSet(dataSetScaled,n_steps_in,n_steps_out)
print(X.shape)
print(y.shape)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

n_features = x_train.shape[2]
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(200, input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(x_train, y_train, epochs=50,validation_data=(x_test,y_test) ,verbose=1)



In [ ]:
predicData = model.predict(x_test, verbose=1)
predictlist20days=[]
for i in range(len(predicData[0])):
    predictlist20days.append(predicData[0][i][-1])
days120list=[]
for i in range(len(x_test[0])):
    days120list.append(x_test[0][i][-1])

days120list20days=[]
for i in range(len(y_test[0])):
    days120list20days.append(y_test[0][i][-1])
predictlist20days

In [ ]:
aAxis=np.arange(0,len(days120list))
bAxis=np.arange(len(days120list)-1,len(days120list)+19)
plt.plot(aAxis,days120list)
plt.plot(bAxis,days120list20days,color='cyan')
plt.plot(bAxis,predictlist20days,color='red')

# REFERENCES

* Groundwater Prediction
  * Changhuia.P,Il-Moon.C(2020). Evaluating the groundwater prediction using LSTM model. J. Korea Water Resour. Assoc. Vol. 53, No. 4, pp. 273-283



* EDA (Exploratory Data Analysis)
  - https://www.kaggle.com/luca31394/acea-eda-and-data-cleaning-deep-water/output#0.-Geographic-overview-of-the-sites
  - https://www.kaggle.com/iamleonie/eda-quenching-the-thirst-for-insights

* Times Series Analysis
  - https://www.kaggle.com/andreshg/timeseries-analysis-a-complete-guide
  - 
* LSTM(Long-short Term Memory)
  - https://www.kaggle.com/iwasnothing/river-level-detection-using-lstm#
  - https://dgkim5360.tistory.com/entry/understanding-long-short-term-memory-lstm-kr

* VAR(Vector Autoregression Model) - https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/